Using the good old MNIST digits, we'll explore hyper parameters. First up -- let's load and normalize the digits.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Dropout, Flatten, MaxPooling2D, Conv2D, BatchNormalization, ZeroPadding2D, Reshape
from keras.models import Model, Sequential
import numpy as np
from sklearn.model_selection import GridSearchCV, PredefinedSplit
import keras.wrappers.scikit_learn

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
train_images = np.expand_dims(x_train / np.max(x_train), -1)
test_images = np.expand_dims(x_test / np.max(x_test), -1)
train_labels = keras.utils.to_categorical(y_train, 10)
test_labels = keras.utils.to_categorical(y_test, 10)
train_images.shape, train_labels.shape

((60000, 28, 28, 1), (60000, 10))

Scikit learn has a great 'standardized' interface that we can use with keras. This requires a 'build function' -- which creates a model.

This build function has a set of parameters -- hyperparameters -- that we will vary in order to find a 'best' model.

In [4]:
input_shape = train_images[0].shape
num_classes = 10

def builder(activation='relu', depth=1, pool=3, 
            strides=2, filters=32, hidden=64, dropout=0.0,
            loss='categorical_crossentropy', optimzer='adam'):
    model = Sequential()
    # initial reshape to have consistent layering
    model.add(Reshape(input_shape, input_shape=input_shape))
    
    # convolutional stack
    for i in range(depth):
        model.add(Conv2D(filters, pool, activation=activation))
        model.add(ZeroPadding2D(pool//2))
        model.add(MaxPooling2D(pool, strides=strides))
        if dropout > 0:
            model.add(Dropout(dropout))
    
    # multilayer perceptron
    model.add(Flatten())
    for i in range(depth):
        model.add(Dense(hidden, activation=activation))
        if dropout > 0:
            model.add(Dropout(dropout))
    # final class activation
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(loss=loss, optimizer=optimzer, metrics=['accuracy'])
    return model

classifier = keras.wrappers.scikit_learn.KerasClassifier(builder)

Now -- a simple model run like we are used to -- but notice we are just training.

In [5]:
classifier.fit(train_images, train_labels)

Epoch 1/1
60000/60000 [==============================] - 17s 291us/step - loss: 0.1558 - acc: 0.9542


Interesting, that just ran one epoch. Epochs -- that itself is a hyperparameter. Let's set up a grid search to look through some epoch values.

Here is where we introduce *grid search*. It's not the most efficient mechanism, as it is pretty exhaustive, but it gets the job done if you have a lot of compute power. Essentially, think of a big checkerboard -- which can be more than two dimensions -- and every possible parameter permutation is tested, saving you from doing so by hand.

Since we already have separated testing data, we'll use a custom `cv` to split at the index point where the training images end, and then just concatenate the training and testing data end to end. `GridSearchCV` will split it back apart for us!

The `PredefinedSplit` let's us declare which samples are training `-1` and testing folds. We'll use only one testing fold, with a value of `0`.

We're setting refit to false. The is no real need for us to train with all our data when we are looking for just the parameters. However, in a real application setting, where your data to predict will be *in the future* using all your training data to improve the model can help.

In [6]:
param_grid = {
    'epochs': [1, 2]
}

all_images = np.concatenate((train_images, test_images), axis=0)
all_labels = np.concatenate((train_labels, test_labels), axis=0)
train_test_bitmap = np.concatenate(
    (np.full(len(train_images), -1), np.zeros(len(test_images))),
    axis=0)

grid = GridSearchCV(estimator=classifier, 
                    param_grid=param_grid, 
                    refit=False,
                    verbose=1,
                    cv=PredefinedSplit(train_test_bitmap))
grid_result = grid.fit(all_images, all_labels)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 1 folds for each of 2 candidates, totalling 2 fits
Epoch 1/1
60000/60000 [==============================] - 5s 82us/step
Epoch 1/2
60000/60000 [==============================] - 12s 200us/step - loss: 0.1479 - acc: 0.9563
Epoch 2/2
60000/60000 [==============================] - 5s 78us/step
Best: 0.985900 using {'epochs': 2}


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   49.7s finished


We can see while this runs -- from Keras progress bar -- the direct accuracy but not the validation accuracy. That's the role of the splits, where we will use the validation data -- the part bitmapped to 0 -- to extract the accuracy.

From there -- our final output is -- the best set of parameters.

OK. That was a very simple 'grid', just two squares to search. Now, let's explore a much larger space. Just a warning, this is going to take a **long while to run**. On my system, I did this with a GPU, but if you don't have a GPU, you can set `n_jobs` to have your CPUs go at this work in parallel.

Ultimately -- this is what folks mean when they say *caviar*. We're going to try a lot of things in parallel and see what works, just like a fish laying eggs!


In [7]:
param_grid = {
    'epochs': [4, 8, 16],
    'activation': ['relu', 'elu'],
    'depth': [1, 2, 4],
    'filters': [32, 64, 128],
    'hidden': [64, 128, 256],
    'dropout': [0.0, 0.25, 0.5]
}
grid = GridSearchCV(estimator=classifier, 
                    param_grid=param_grid, 
                    refit=False,
                    verbose=1,
                    cv=PredefinedSplit(train_test_bitmap))
grid_result = grid.fit(all_images, all_labels)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 1 folds for each of 486 candidates, totalling 486 fits
Epoch 1/4
60000/60000 [==============================] - 12s 208us/step - loss: 0.1597 - acc: 0.9515
Epoch 2/4
60000/60000 [==============================] - 12s 199us/step - loss: 0.0566 - acc: 0.9829
Epoch 3/4
60000/60000 [==============================] - 11s 188us/step - loss: 0.0404 - acc: 0.9875
Epoch 4/4
60000/60000 [==============================] - 5s 79us/step
Epoch 1/4
60000/60000 [==============================] - 12s 203us/step - loss: 0.1507 - acc: 0.9542
Epoch 2/4
60000/60000 [==============================] - 11s 192us/step - loss: 0.0517 - acc: 0.9840
Epoch 3/4
60000/60000 [==============================] - 12s 197us/step - loss: 0.0349 - acc: 0.9892
Epoch 4/4
60000/60000 [==============================] - 5s 77us/step
Epoch 1/4
60000/60000 [==============================] - 13s 212us/step - loss: 0.1221 - acc: 0.9620
Epoch 2/4
60000/60000 [==============================] - 12s 202us/step - loss: 0.0416 - a

60000/60000 [==============================] - 13s 211us/step - loss: 0.0102 - acc: 0.9969
Epoch 8/8
60000/60000 [==============================] - 5s 90us/step
Epoch 1/8
60000/60000 [==============================] - 14s 231us/step - loss: 0.1311 - acc: 0.9601
Epoch 2/8
60000/60000 [==============================] - 13s 211us/step - loss: 0.0467 - acc: 0.9857
Epoch 3/8
60000/60000 [==============================] - 13s 215us/step - loss: 0.0296 - acc: 0.9902
Epoch 4/8
60000/60000 [==============================] - 13s 215us/step - loss: 0.0208 - acc: 0.9931
Epoch 5/8
60000/60000 [==============================] - 13s 216us/step - loss: 0.0141 - acc: 0.9954
Epoch 6/8
60000/60000 [==============================] - 13s 217us/step - loss: 0.0109 - acc: 0.9962
Epoch 7/8
60000/60000 [==============================] - 13s 219us/step - loss: 0.0095 - acc: 0.9969
Epoch 8/8
60000/60000 [==============================] - 5s 90us/step
Epoch 1/8
60000/60000 [==============================] - 14s 2

60000/60000 [==============================] - 13s 223us/step - loss: 0.0033 - acc: 0.9991
Epoch 16/16
60000/60000 [==============================] - 5s 91us/step
Epoch 1/16
60000/60000 [==============================] - 14s 236us/step - loss: 0.1277 - acc: 0.9612
Epoch 2/16
60000/60000 [==============================] - 13s 222us/step - loss: 0.0448 - acc: 0.9857
Epoch 3/16
60000/60000 [==============================] - 13s 222us/step - loss: 0.0286 - acc: 0.9909
Epoch 4/16
60000/60000 [==============================] - 13s 224us/step - loss: 0.0184 - acc: 0.9941
Epoch 5/16
60000/60000 [==============================] - 13s 223us/step - loss: 0.0140 - acc: 0.9955
Epoch 6/16
60000/60000 [==============================] - 13s 224us/step - loss: 0.0103 - acc: 0.9967
Epoch 7/16
60000/60000 [==============================] - 13s 222us/step - loss: 0.0087 - acc: 0.9972
Epoch 8/16
60000/60000 [==============================] - 13s 222us/step - loss: 0.0062 - acc: 0.9980
Epoch 9/16
60000/6000

60000/60000 [==============================] - 13s 224us/step - loss: 0.0066 - acc: 0.9980
Epoch 10/16
60000/60000 [==============================] - 13s 214us/step - loss: 0.0049 - acc: 0.9984
Epoch 11/16
60000/60000 [==============================] - 13s 224us/step - loss: 0.0051 - acc: 0.9985
Epoch 12/16
60000/60000 [==============================] - 13s 221us/step - loss: 0.0054 - acc: 0.9983
Epoch 13/16
60000/60000 [==============================] - 13s 224us/step - loss: 0.0044 - acc: 0.9984
Epoch 14/16
60000/60000 [==============================] - 14s 225us/step - loss: 0.0040 - acc: 0.9986
Epoch 15/16
60000/60000 [==============================] - 13s 224us/step - loss: 0.0034 - acc: 0.9990
Epoch 16/16
60000/60000 [==============================] - 6s 97us/step
Epoch 1/16
60000/60000 [==============================] - 14s 227us/step - loss: 0.1204 - acc: 0.9638
Epoch 2/16
60000/60000 [==============================] - 14s 226us/step - loss: 0.0428 - acc: 0.9864
Epoch 3/16
6000

60000/60000 [==============================] - 17s 283us/step - loss: 0.1632 - acc: 0.9508
Epoch 2/4
60000/60000 [==============================] - 15s 257us/step - loss: 0.0660 - acc: 0.9797
Epoch 3/4
60000/60000 [==============================] - 15s 258us/step - loss: 0.0479 - acc: 0.9847
Epoch 4/4
60000/60000 [==============================] - 7s 110us/step
Epoch 1/4
60000/60000 [==============================] - 19s 317us/step - loss: 0.1310 - acc: 0.9589
Epoch 2/4
60000/60000 [==============================] - 17s 280us/step - loss: 0.0517 - acc: 0.9838
Epoch 3/4
60000/60000 [==============================] - 17s 279us/step - loss: 0.0366 - acc: 0.9881
Epoch 4/4
60000/60000 [==============================] - 7s 111us/step
Epoch 1/8
60000/60000 [==============================] - 17s 284us/step - loss: 0.2205 - acc: 0.9341
Epoch 2/8
60000/60000 [==============================] - 15s 253us/step - loss: 0.0930 - acc: 0.9710
Epoch 3/8
60000/60000 [==============================] - 15s

...

60000/60000 [==============================] - 25s 420us/step - loss: 0.1741 - acc: 0.9519
Epoch 3/4
60000/60000 [==============================] - 25s 422us/step - loss: 0.1359 - acc: 0.9632
Epoch 4/4
60000/60000 [==============================] - 13s 213us/step
Epoch 1/4
60000/60000 [==============================] - 35s 576us/step - loss: 0.3259 - acc: 0.8959
Epoch 2/4
60000/60000 [==============================] - 26s 426us/step - loss: 0.1272 - acc: 0.9641
Epoch 3/4
60000/60000 [==============================] - 26s 433us/step - loss: 0.1066 - acc: 0.9690
Epoch 4/4
60000/60000 [==============================] - 13s 218us/step
Epoch 1/4
60000/60000 [==============================] - 34s 571us/step - loss: 0.7294 - acc: 0.7535
Epoch 2/4
60000/60000 [==============================] - 25s 419us/step - loss: 0.3434 - acc: 0.8951
Epoch 3/4
60000/60000 [==============================] - 25s 425us/step - loss: 0.2958 - acc: 0.9088
Epoch 4/4
60000/60000 [==============================] - 1

60000/60000 [==============================] - 37s 609us/step - loss: 0.5974 - acc: 0.8014
Epoch 2/8
60000/60000 [==============================] - 27s 444us/step - loss: 0.2674 - acc: 0.9180
Epoch 3/8
60000/60000 [==============================] - 26s 438us/step - loss: 0.2282 - acc: 0.9309
Epoch 4/8
60000/60000 [==============================] - 26s 438us/step - loss: 0.2048 - acc: 0.9369
Epoch 5/8
60000/60000 [==============================] - 27s 443us/step - loss: 0.1883 - acc: 0.9430
Epoch 6/8
60000/60000 [==============================] - 26s 437us/step - loss: 0.1783 - acc: 0.9458
Epoch 7/8
60000/60000 [==============================] - 26s 434us/step - loss: 0.1709 - acc: 0.9481
Epoch 8/8
60000/60000 [==============================] - 14s 228us/step
Epoch 1/8
60000/60000 [==============================] - 36s 599us/step - loss: 0.4467 - acc: 0.8588
Epoch 2/8
60000/60000 [==============================] - 26s 441us/step - loss: 0.1763 - acc: 0.9502
Epoch 3/8
60000/60000 [======

60000/60000 [==============================] - 27s 444us/step - loss: 0.3067 - acc: 0.9115
Epoch 3/16
60000/60000 [==============================] - 27s 445us/step - loss: 0.2469 - acc: 0.9265
Epoch 4/16
60000/60000 [==============================] - 27s 444us/step - loss: 0.2188 - acc: 0.9365
Epoch 5/16
60000/60000 [==============================] - 27s 454us/step - loss: 0.1966 - acc: 0.9435
Epoch 6/16
60000/60000 [==============================] - 27s 443us/step - loss: 0.1878 - acc: 0.9459
Epoch 7/16
60000/60000 [==============================] - 27s 447us/step - loss: 0.1790 - acc: 0.9485
Epoch 8/16
60000/60000 [==============================] - 27s 445us/step - loss: 0.1743 - acc: 0.9484
Epoch 9/16
60000/60000 [==============================] - 27s 449us/step - loss: 0.1616 - acc: 0.9526
Epoch 10/16
60000/60000 [==============================] - 27s 451us/step - loss: 0.1584 - acc: 0.9535
Epoch 11/16
60000/60000 [==============================] - 27s 442us/step - loss: 0.1620 - a

60000/60000 [==============================] - 27s 451us/step - loss: 0.0816 - acc: 0.9785
Epoch 14/16
60000/60000 [==============================] - 27s 442us/step - loss: 0.0803 - acc: 0.9792
Epoch 15/16
60000/60000 [==============================] - 27s 448us/step - loss: 0.0747 - acc: 0.9806
Epoch 16/16
60000/60000 [==============================] - 14s 232us/step
Epoch 1/16
60000/60000 [==============================] - 38s 640us/step - loss: 0.2971 - acc: 0.9071
Epoch 2/16
60000/60000 [==============================] - 27s 447us/step - loss: 0.1164 - acc: 0.9677
Epoch 3/16
60000/60000 [==============================] - 27s 453us/step - loss: 0.0999 - acc: 0.9719
Epoch 4/16
60000/60000 [==============================] - 27s 451us/step - loss: 0.0889 - acc: 0.9758
Epoch 5/16
60000/60000 [==============================] - 27s 454us/step - loss: 0.0825 - acc: 0.9775
Epoch 6/16
60000/60000 [==============================] - 28s 459us/step - loss: 0.0787 - acc: 0.9785
Epoch 7/16
60000/

ValueError: Negative dimension size caused by subtracting 3 from 2 for 'conv2d_250/convolution' (op: 'Conv2D') with input shapes: [?,2,2,32], [3,3,32,32].

Machine learning is often a lot of human waiting, but it beats sitting and manually supervising experiments!